In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 200)

import numpy as np
import re

# for NLP related tasks
import spacy
nlp=spacy.load('en_core_web_sm',disable=["tagger", "parser","ner"])

In [2]:
!pip install xgboost

# Loading and Exploring Data

In [3]:
df = pd.read_csv(r'C:\Users\DAG9KOR\Downloads\Tweets.csv')
print('Shape -->',df.shape)
df.head()

Shape --> (14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
# Some sample tweets
df['text'].sample(5)

7018                                                                                               “@JetBlue: Our fleet's on fleek. http://t.co/cRFrwpc1Sx”😂😂😂
13175                         @AmericanAir That will be the third time I have been called by 800-433-7300 an hung on before anyone speaks. What do I do now???
8297             @JetBlue I can't do that flight. I need a Late Flightr one! I need you to change my flight. You guys changed it and now I can't do that time!
10626             @USAirways Cool. Now @SweetingR has been told you can't re-route his bags after he was told to leave them last night. He needs them to race.
14266    @AmericanAir been a very long day in Philly...12:13pm flt to ORD Cancelled Flighted with NO notice, no message, no email.  NOT NICE FOR WE PLATINUMS.
Name: text, dtype: object

In [5]:
# class distribution
df['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [6]:
# class distribution in percentage
df['airline_sentiment'].value_counts(normalize = True)*100

negative    62.691257
neutral     21.168033
positive    16.140710
Name: airline_sentiment, dtype: float64

# Text Cleaning

In [7]:
#define a function for text cleaning
def text_cleaner(text):
  
  #remove user mentions
    text = re.sub(r'@[A-Za-z0-9]+','',text)           
  
  #remove hashtags
  #text = re.sub(r'#[A-Za-z0-9]+','',text)         
  
  #remove links
    text = re.sub(r'http\S+', '', text)  

  #convering text to lower case
    text = text.lower()

  # fetch only words
    text = re.sub("[^a-z]+", " ", text)

  # removing extra spaces
    text=re.sub("[\s]+"," ",text)
  
  # creating doc object
    doc=nlp(text)

  # remove stopwords and lemmatize the text
    tokens=[token.lemma_ for token in doc if(token.is_stop==False)]
  
  #join tokens by space
    return " ".join(tokens)

In [8]:
# perform text cleaning
df['clean_text']= df['text'].apply(text_cleaner)

C:\Users\DAG9KOR\Anaconda3_latest\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [9]:
# save cleaned text and labels to a variable
text   = df['clean_text'].values
labels = df['airline_sentiment'].values

In [10]:
# Sample cleaned text
text[:10]

array(['  said', '  plus ve added commercials experience tacky',
       '  didn t today mean need trip',
       '  s aggressive blast obnoxious entertainment guests faces amp little recourse',
       '  s big bad thing',
       '  seriously pay flight seats didn t playing s bad thing flying va',
       '  yes nearly time fly vx ear worm won t away',
       '  missed prime opportunity men hats parody', '  didn t d',
       '  amazing arrived hour early good'], dtype=object)

In [11]:
# Sample labels
labels[:10]

array(['neutral', 'positive', 'neutral', 'negative', 'negative',
       'negative', 'positive', 'neutral', 'positive', 'positive'],
      dtype=object)

# Data Preparation

## Label Encoding

In [12]:
#importing label encoder
from sklearn.preprocessing import LabelEncoder

#define label encoder
le = LabelEncoder()

#fit and transform target strings to a numbers
labels = le.fit_transform(labels)

In [13]:
# Sample labels
labels[:10]

array([1, 2, 1, 0, 0, 0, 2, 1, 2, 2])

In [14]:
# Meaning of each label
le.inverse_transform([0,1,2])

array(['negative', 'neutral', 'positive'], dtype=object)

## Split Data

In [15]:
from sklearn.model_selection import train_test_split

# Splitting into train and validation set
x_train,x_val,y_train,y_val=train_test_split(text, labels,stratify=labels, test_size=0.2, random_state=0,shuffle=True)

In [16]:
print('x_train:',x_train.shape,'y_train:',y_train.shape)
print('x_val:',x_val.shape,'y_val:',y_val.shape)

x_train: (11712,) y_train: (11712,)
x_val: (2928,) y_val: (2928,)


## Feature Engineering using TF-IDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
# initialize TFIDF
word_vectorizer = TfidfVectorizer(max_features=1000)

In [19]:
# Fitting Vectorizer on Train set
word_vectorizer.fit(x_train)

TfidfVectorizer(max_features=1000)

In [20]:
# create TF-IDF vectors for Train Set
train_word_features = word_vectorizer.transform(x_train)
train_word_features

<11712x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 65698 stored elements in Compressed Sparse Row format>

In [21]:
# create TF-IDF vectors for Validation Set
val_word_features = word_vectorizer.transform(x_val)
val_word_features

<2928x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 16555 stored elements in Compressed Sparse Row format>

# Model Building

## Naive Bayes

In [22]:
# Importing for modeling
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

In [23]:
# Training model
nb_model=MultinomialNB().fit(train_word_features,y_train)
nb_model

MultinomialNB()

In [24]:
# Make predictions for train set
train_pred_nb=nb_model.predict(train_word_features)

In [25]:
train_pred_nb

array([2, 0, 2, ..., 2, 0, 0])

In [26]:
# Evaluating on Training Set
print("F1-score on Train Set:",f1_score(y_train,train_pred_nb,average="weighted"))
train_nb_f1 = f1_score(y_train,train_pred_nb,average="weighted")

F1-score on Train Set: 0.7303639422112124


In [27]:
# Make predictions for validation set
val_pred_nb=nb_model.predict(val_word_features)

# Evaluating on Validation Set
print("F1-score on Validation Set:",f1_score(y_val,val_pred_nb,average="weighted"))
val_nb_f1 = f1_score(y_val,val_pred_nb,average="weighted")

F1-score on Validation Set: 0.6885748623606444


## Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
# Training model
lr_model=LogisticRegression().fit(train_word_features,y_train)
lr_model

C:\Users\DAG9KOR\Anaconda3_latest\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
# Make predictions for train set
train_pred_lr=lr_model.predict(train_word_features)
train_pred_nb

array([2, 0, 2, ..., 2, 0, 0])

In [31]:
# Evaluating on Training Set
print("F1-score on Train Set:",f1_score(y_train,train_pred_lr,average="weighted"))
train_lr_f1 = f1_score(y_train,train_pred_lr,average="weighted")

F1-score on Train Set: 0.8076448633688


In [32]:
# Make predictions for validation set
val_pred_lr=lr_model.predict(val_word_features)

# Evaluating on Validation Set
print("F1-score on Validation Set:",f1_score(y_val,val_pred_lr,average="weighted"))
val_lr_f1 = f1_score(y_val,val_pred_lr,average="weighted")

F1-score on Validation Set: 0.7531280504280244


## Model Building Summary
|        Model        | Train Set | Validation Set |
|:-------------------:|:---------:|:--------------:|
|     Naive Bayes     |   0.7274  |     0.6791     |
| Logistic Regression |   0.8089  |     0.7598     |

 Logistic Regression performs better than Naive Bayes on this dataset.

# Final Sentiment Analysis Pipeline

In [33]:
def sentiment_analyzer(tweet):
  # Cleaning Tweet
    cleaned_tweet=text_cleaner(tweet)

  # Feature Engineering
    tweet_vector=word_vectorizer.transform([cleaned_tweet])

  # Predicting Sentiment
    label=lr_model.predict(tweet_vector)

    return le.inverse_transform(np.array(label))

<font size=4>**Sample Tweet:**</font>
<p>@USAirways flt 419. 2+ hrs Late Flight, baggage + 1 more hr. Now I see they delivered my suitcase wet inside &amp; out. #NotHappy</p>

In [34]:
sentiment_analyzer("@USAirways flt 419. 2+ hrs Late Flight, baggage + 1 more hr. Now I see they delivered my suitcase wet inside &amp; out. #NotHappy")

C:\Users\DAG9KOR\Anaconda3_latest\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


array(['negative'], dtype=object)

In [35]:
sentiment_analyzer("@VirginAmerica I love the hipster innovation. You are a feel good brand.")

array(['positive'], dtype=object)

In [36]:
sentiment_analyzer("I am not happy with the service since it is very horrible and the prices are very high")

array(['negative'], dtype=object)

In [37]:
sentiment_analyzer("I love the hospitality of the air hostess and like to take the service once again")

array(['positive'], dtype=object)

In [38]:
sentiment_analyzer("i like the service but the price is little expensive")

array(['negative'], dtype=object)

## XGBoost

In [39]:
import xgboost as xgb

In [40]:
xgb_cl = xgb.XGBClassifier()

In [41]:
xgb_cl.fit(train_word_features,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [42]:
preds_val = xgb_cl.predict(val_word_features)
preds_train = xgb_cl.predict(train_word_features)

In [43]:
print("F1-score on Train Set:",f1_score(y_train,preds_train,average="weighted"))
print("F1-score on Validation Set:",f1_score(y_val,preds_val,average="weighted"))

train_xg_f1 = f1_score(y_train,preds_train,average="weighted")
val_xg_f1 = f1_score(y_val,preds_val,average="weighted")

F1-score on Train Set: 0.8088068831380627
F1-score on Validation Set: 0.7071293939384495


## Linear SVC

In [44]:
from sklearn.svm import LinearSVC

In [45]:
lsvc = LinearSVC(verbose=0)

In [46]:
lsvc.fit(train_word_features,y_train)

LinearSVC()

In [47]:
preds_val_lsvc = lsvc.predict(val_word_features)
preds_train_lsvc = lsvc.predict(train_word_features)

In [48]:
print("F1-score on Train Set:",f1_score(y_train,preds_train_lsvc,average="weighted"))
print("F1-score on Validation Set:",f1_score(y_val,preds_val_lsvc,average="weighted"))

train_lsvc_f1 = f1_score(y_train,preds_train_lsvc,average="weighted")
val_lsvc_f1 = f1_score(y_val,preds_val_lsvc,average="weighted")

F1-score on Train Set: 0.8203590025407713
F1-score on Validation Set: 0.7555303049855076


In [49]:
f1_df = {"model":['Naive Bayes','Logistic Regression','XGboost','Linear SVC'],'train_F1_score':[train_nb_f1,train_lr_f1,train_xg_f1,train_lsvc_f1],
         'val_F1_score':[val_nb_f1,val_lr_f1,val_xg_f1,val_lsvc_f1]}

In [50]:
model_df = pd.DataFrame(f1_df)
model_df

,model,train_F1_score,val_F1_score
0,Naive Bayes,0.730364,0.688575
1,Logistic Regression,0.807645,0.753128
2,XGboost,0.808807,0.707129
3,Linear SVC,0.820359,0.755530
